<a href="https://colab.research.google.com/github/aditya112346/Assignment-Zeotap-/blob/main/Aditya_Garg_Lookalike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Prepare customer and transaction data for modeling
# Aggregating purchase history by customer and product categories
merged_data = transactions.merge(products, on='ProductID').merge(customers, on='CustomerID')
category_sales_per_customer = merged_data.groupby(['CustomerID', 'Category'])['TotalValue'].sum().unstack(fill_value=0)

# Merge with customer details
customer_data = customers.set_index('CustomerID').join(category_sales_per_customer, how='left').fillna(0)

# Scale the data for similarity calculations
scaler = StandardScaler()
customer_data_scaled = scaler.fit_transform(customer_data.iloc[:, 3:])

# Calculate cosine similarity between customers
similarity_matrix = cosine_similarity(customer_data_scaled)
similarity_df = pd.DataFrame(similarity_matrix, index=customer_data.index, columns=customer_data.index)

# Generate lookalike recommendations for the first 20 customers
lookalike_map = {}
for customer_id in customer_data.index[:20]:
    similar_customers = similarity_df[customer_id].sort_values(ascending=False).iloc[1:4]
    lookalike_map[customer_id] = list(zip(similar_customers.index, similar_customers.values))

# Convert to Lookalike.csv format
lookalike_df = pd.DataFrame({
    'CustomerID': [customer for customer, recs in lookalike_map.items() for _ in recs],
    'LookalikeID': [rec[0] for recs in lookalike_map.values() for rec in recs],
    'Score': [rec[1] for recs in lookalike_map.values() for rec in recs]
})

# Save the result
lookalike_csv_path = '/content/Lookalike.csv'
lookalike_df.to_csv(lookalike_csv_path, index=False)

lookalike_csv_path

'/content/Lookalike.csv'